Sacar mi historial de partidas SUMMONER-V4, MATCH-V5
o 
info de los jugadores de una división concreta LEAGUE-V4 division

In [1]:
import requests
import pandas as pd
from creds import api_key
import re

In [2]:
url_summoner = 'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Masgt7'

Nos centramos en SUMMONER-V4que tiene que ver con cualquier cosa que haga una cuenta en concreto

In [3]:
url_summoner = url_summoner + '?api_key=' + api_key

In [4]:
r = requests.get (url_summoner)
r.status_code

200

In [5]:
player_info = r.json()
print (player_info)

{'id': '7xKp5xDyqk1u0_RKrh8SdaSHYhjpThiq7XXJscT2DQx273I', 'accountId': 'NcGfPSobpg1gNOsZBjrQNCVyeIg7MyijVoglw0WBQioOST0', 'puuid': 'uFIekhnHnZjaIXG_IG5y2Kb_UbBsEv57LksBUe6qMh0h0MPBKg8xoWtsbQkvXcPhlHLNGn8xZwqzGQ', 'name': 'Masgt7', 'profileIconId': 940, 'revisionDate': 1700743850091, 'summonerLevel': 74}


Obtenida la info de mi cuenta, ahora hacemos otra request para obtener identificadores de mis últimas partidas a partir del 'puuid'

In [6]:
puuid = player_info['puuid']

In [7]:
url_match_history = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/' + puuid + '/ids?type=normal&start=0&count=20&api_key=' + api_key #Limitamos la lista de partidas a 20
r = requests.get(url_match_history)

In [8]:
if r.status_code == 200:
    game_id = r.json()
    print (game_id)
else:
    print ('Error', r.status_code, ':',)  

['EUW1_6688283685', 'EUW1_6688256964', 'EUW1_6688236897', 'EUW1_6688218767', 'EUW1_6686773929', 'EUW1_6686744190', 'EUW1_6684157765', 'EUW1_6684133215', 'EUW1_6682813114', 'EUW1_6682767095', 'EUW1_6682519608', 'EUW1_6678781224', 'EUW1_6678748896', 'EUW1_6678696426', 'EUW1_6678677984', 'EUW1_6678647009', 'EUW1_6677476957', 'EUW1_6677447766', 'EUW1_6673723132', 'EUW1_6673680411']


Vamos a sacar ahora información de cada partida y almacenarla en ficheros .csv por separado

In [9]:
counter = 1
for i in game_id:
    url_game_info = 'https://europe.api.riotgames.com/lol/match/v5/matches/' + i + '?api_key=' + api_key
    game_info = requests.get(url_game_info)
    index_player = game_info.json()['metadata']['participants'].index(puuid)
    game_data = game_info.json()['info']['participants'][index_player]
    game_df = pd.json_normalize(game_data)
    game_df.to_csv ('games\\game' + str(counter) + '.csv', sep = ',', mode = 'x')
    counter += 1

Cargamos una lista con DF a partir de los CSV

In [10]:
list_df = []

for i in range(len(game_id)):
    list_df.append((pd.read_csv('games\\game' + str (i+1) + '.csv')))
    print (list_df[i].shape)

(1, 262)
(1, 264)
(1, 264)
(1, 264)
(1, 264)
(1, 263)
(1, 268)
(1, 269)
(1, 268)
(1, 269)
(1, 269)
(1, 268)
(1, 268)
(1, 267)
(1, 267)
(1, 267)
(1, 267)
(1, 266)
(1, 268)
(1, 268)


No tienen las mismas columnas, de modo que vamos a quedarnos únicamente con las que tengan en común

In [11]:
common_columns = list(set.intersection(*(set(df.columns) for df in list_df)))
common_columns[0:10] #Vemos algunas

['playerScore5',
 'challenges.epicMonsterKillsNearEnemyJungler',
 'totalUnitsHealed',
 'challenges.immobilizeAndKillWithAlly',
 'challenges.buffsStolen',
 'missions.playerScore1',
 'missions.playerScore3',
 'challenges.turretPlatesTaken',
 'challenges.killsWithHelpFromEpicMonster',
 'totalTimeCCDealt']

In [12]:
df_joined = pd.concat(list_df)
df_joined = df_joined [common_columns]

df_joined.head()

,playerScore5,challenges.epicMonsterKillsNearEnemyJungler,totalUnitsHealed,challenges.immobilizeAndKillWithAlly,challenges.buffsStolen,missions.playerScore1,missions.playerScore3,challenges.turretPlatesTaken,challenges.killsWithHelpFromEpicMonster,totalTimeCCDealt,...,timePlayed,challenges.turretTakedowns,totalDamageDealtToChampions,challenges.scuttleCrabKills,unrealKills,challenges.gameLength,missions.playerScore8,challenges.tookLargeDamageSurvived,playerScore0,champExperience
0,0,0,1,16,0,0,0,0,0,153,...,1419,0,25707,0,0,1419.965720,0,2,0,22871
0,0,0,1,8,0,0,0,0,0,12,...,1164,0,27617,0,0,1164.431252,0,0,0,19144
0,0,0,1,8,0,0,0,0,0,55,...,1387,3,26658,0,0,1388.050799,0,1,0,23613
0,0,0,1,8,0,0,0,0,0,57,...,992,3,13649,0,0,992.416079,0,0,0,15126
0,0,0,1,1,0,0,0,0,0,135,...,1260,2,36196,0,0,1260.134893,0,0,0,20153


Eliminamos las palabras previas al punto

In [13]:
def remove_dot (string):
    pattern = r'.*?\.(.*)'
    result = re.match(pattern, string)
    if result:
        return result.group(1)
    else:
        return string

In [14]:
df_columns = list(df_joined.columns)

for i,j in enumerate(df_columns):
    df_columns[i] = remove_dot(j)

df_joined.columns = df_columns

df_joined.head(1)

,playerScore5,epicMonsterKillsNearEnemyJungler,totalUnitsHealed,immobilizeAndKillWithAlly,buffsStolen,playerScore1,playerScore3,turretPlatesTaken,killsWithHelpFromEpicMonster,totalTimeCCDealt,...,timePlayed,turretTakedowns,totalDamageDealtToChampions,scuttleCrabKills,unrealKills,gameLength,playerScore8,tookLargeDamageSurvived,playerScore0,champExperience
0,0,0,1,16,0,0,0,0,0,153,...,1419,0,25707,0,0,1419.96572,0,2,0,22871


In [15]:
df_joined.shape

(20, 260)

We have a very high dimensionality in our dataset, so now we'll check the descriptive statistics of each variable, and we'll drop the columns that we consider not relevant

In [16]:
# display(common_rows)
pd.set_option('display.max_columns', None) #Quitamos el límite de columnas para poder ver todas y evaluar cuáles son importantes
display(df_joined.describe())

,playerScore5,epicMonsterKillsNearEnemyJungler,totalUnitsHealed,immobilizeAndKillWithAlly,buffsStolen,playerScore1,playerScore3,turretPlatesTaken,killsWithHelpFromEpicMonster,totalTimeCCDealt,fullTeamTakedown,magicDamageDealtToChampions,championId,flawlessAces,item5,killingSprees,totalEnemyJungleMinionsKilled,playerAugment1,playerScore7,inhibitorTakedowns,outnumberedNexusKill,dangerPings,largestMultiKill,nexusKills,hadOpenNexus,playerScore8,spell3Casts,jungleCsBefore10Minutes,damageDealtToBuildings,summoner1Id,outerTurretExecutesBefore10Minutes,spell1Casts,physicalDamageDealtToChampions,playerScore9,assistMePings,totalHeal,playerScore5,holdPings,playerSubteamId,totalTimeSpentDead,visionScore,12AssistStreakCount,doubleAces,killParticipation,spell4Casts,multiTurretRiftHeraldCount,profileIcon,twoWardsOneSweeperCount,basicPings,magicDamageDealt,elderDragonKillsWithOpposingSoul,enemyVisionPings,mejaisFullStackInTime,epicMonsterKillsWithin30SecondsOfSpawn,allInPings,multiKillOneSpell,killingSprees,tripleKills,perfectGame,playerScore11,neutralMinionsKilled,champLevel,takedownsAfterGainingLevelAdvantage,visionClearedPings,objectivesStolenAssists,enemyChampionImmobilizations,itemsPurchased,onMyWayPings,placement,turretTakedowns,enemyJungleMonsterKills,summonerLevel,landSkillShotsEarlyGame,effectiveHealAndShielding,perfectDragonSoulsTaken,takedownsInAlcove,teamDamagePercentage,getBackPings,item3,detectorWardsPlaced,visionScorePerMinute,inhibitorsLost,multikills,damagePerMinute,timeCCingOthers,getTakedownsInAllLanesEarlyJungleAsLaner,initialBuffCount,item4,trueDamageDealt,killsOnRecentlyHealedByAramPack,trueDamageDealtToChampions,playerScore0,dancedWithRiftHerald,stealthWardsPlaced,summoner2Id,killAfterHiddenWithAlly,maxKillDeficit,teamBaronKills,legendaryCount,totalHealsOnTeammates,doubleKills,subteamPlacement,totalDamageTaken,bountyLevel,controlWardsPlaced,multikillsAfterAggressiveFlash,spell2Casts,survivedThreeImmobilizesInFight,playerScore3,turretKills,alliedJungleMonsterKills,skillshotsDodged,pushPings,playerScore4,outnumberedKills,damageTakenOnTeamPercentage,deathsByEnemyChamps,assists,killsNearEnemyTurret,item2,dragonTakedowns,playerAugment2,totalAllyJungleMinionsKilled,totalDamageShieldedOnTeammates,poroExplosions,wardsPlaced,takedownsFirstXMinutes,playerScore6,unseenRecalls,baronTakedowns,totalMinionsKilled,bountyGold,Unnamed: 0,killsUnderOwnTurret,pickKillWithAlly,takedowns,moreEnemyJungleThanOpponent,participantId,quickSoloKills,commandPings,knockEnemyIntoTeamAndKill,statPerks.offense,dragonKills,epicMonsterSteals,turretsTakenWithRiftHerald,playerScore2,teamId,blastConeOppositeOpponentCount,damageDealtToTurrets,playerScore6,kTurretsDestroyedBeforePlatesFall,acesBefore15Minutes,quickCleanse,skillshotsHit,soloBaronKills,consumablesPurchased,takedownOnFirstTurret,firstTurretKilled,playerScore2,trueDamageTaken,wardsKilled,teamPosition,enemyMissingPings,kills,pentaKills,playerAugment3,quickFirstTurret,statPerks.flex,turretsLost,completeSupportQuestInTime,needVisionPings,inhibitorKills,survivedSingleDigitHpCount,teamElderDragonKills,killedChampTookFullTeamDamageSurvived,initialCrabCount,saveAllyFromDeath,championTransform,killsOnOtherLanesEarlyJungleAsLaner,teamRiftHeraldKills,twentyMinionsIn3SecondsCount,totalDamageDealt,takedownsInEnemyFountain,goldSpent,playerScore10,item6,magicDamageTaken,physicalDamageDealt,takedownsBeforeJungleMinionSpawn,baronKills,visionWardsBoughtInGame,baitPings,quadraKills,wardTakedownsBefore20M,sightWardsBoughtInGame,item1,largestKillingSpree,playerScore7,damageSelfMitigated,physicalDamageTaken,summoner1Casts,goldPerMinute,lostAnInhibitor,deaths,item0,playerScore1,playerScore4,goldEarned,laneMinionsFirst10Minutes,wardTakedowns,abilityUses,playerAugment4,largestCriticalStrike,playerScore11,riftHeraldTakedowns,summoner2Casts,epicMonsterStolenWithoutSmite,playerScore9,statPerks.defense,snowballsHit,objectivesStolen,dodgeSkillShotsSmallWindow,longestTimeSpentLiving,soloKills,nexusLost,nexusTakedowns,elderDra

In [17]:
df_joined.drop(['quickSoloKills', 'survivedSingleDigitHpCount', 'soloBaronKills', 'pushPings','playerScore3','playerScore8', 
                'getTakedownsInAllLanesEarlyJungleAsLaner', 'twentyMinionsIn3SecondsCount', 'playerScore6', 'summoner2Id', 'playerScore1', 'playerScore4', 
                'playerScore9', 'playerScore2', 'mejaisFullStackInTime', 'playerAugment1', 'playerScore1', 'playerAugment4', 'playerSubteamId', 
                'playerScore7','playerScore11', 'summoner1Id', 'playerScore5', 'puuid', 'playerScore6', 'playerScore0', 'summonerLevel', 'playerScore3', 
                'Unnamed: 0', 'subteamPlacement', 'playerScore9', 'championTransform', 'playerScore11', 'playerScore2', 'playerAugment3', 'poroExplosions', 
                'participantId', 'placement', 'teamId', 'quickCleanse', 'profileIcon', 'nexusLost', 'playerScore7', 'playerScore5', 'perfectDragonSoulsTaken', 
                'playerScore10', 'playerAugment2', 'playerScore0', 'playerScore4', 'largestKillingSpree'], axis=1, inplace=True)

In [18]:
df_joined.shape

(20, 208)

In [20]:
df_joined.to_csv('game_history.csv', mode = 'x')